In [ ]:
!pip install tsai

In [ ]:
import pandas as pd
import numpy as np
import gc
import torch
import torch.nn as nn
from tsai.all import *
from fastai.tabular.all import *

In [ ]:
train = pd.read_parquet('../input/ubiquant-parquet/train_low_mem.parquet')

In [ ]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                #if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                #    df[col] = df[col].astype(np.float16)
                #elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                #    df[col] = df[col].astype(np.float32)
                #else:
                df[col] = df[col].astype(np.float16)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [ ]:
train = train[train["time_id"]>600]
train = reduce_mem_usage(train)

In [ ]:
numerical_feature = ["time_id"]+list(train.filter(like="f").columns)

In [ ]:
def pearson_loss(x, y):
    xd = x - x.mean()
    yd = y - y.mean()
    nom = (xd*yd).sum()
    denom = ((xd**2).sum() * (yd**2).sum()).sqrt()
    return 1 - nom / denom

In [ ]:
train.reset_index(drop=True, inplace=True)
valid_idx = list(train[train["time_id"]>1100].index)
dls = TabularDataLoaders.from_df(df=train, cont_names=numerical_feature, y_names="target", 
                                 valid_idx=valid_idx,
                                 bs=1024
                                )
x_cat, x_cont, yb = first(dls.train)
model = TabTransformer(classes={}, cont_names=numerical_feature, c_out=1).cuda()

In [ ]:
learn = Learner(dls, model,loss_func=pearson_loss, metrics = [mse, mae])
learn.fine_tune(3)

In [ ]:
learn.plot_metrics()